In [13]:
import pandas as pd
import numpy as np
import folium
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [11]:
df_venues_raw = pd.read_pickle('app_ldn_venues_raw.pkl')
df_rent = pd.read_pickle('app_ldn_rent.pkl')
ldn_geojson = 'london_boroughs_proper.geojson'
df_groups_norm = pd.read_pickle('app_ldn_groups_norm.pkl')

In [12]:
df_groups_norm

,Borough,Eating out,Entertainment,Going out,Green spaces,Groceries,Health and Sports,Other,Public Transport,Shopping
0,Barking and Dagenham,0.209150,0.058824,0.058824,0.091503,0.091503,0.078431,0.163399,0.071895,0.176471
1,Barnet,0.414861,0.018576,0.066563,0.068111,0.061920,0.061920,0.078947,0.037152,0.191950
2,Bexley,0.296578,0.022814,0.121673,0.034221,0.079848,0.053232,0.098859,0.045627,0.247148
3,Brent,0.400802,0.022044,0.082164,0.054108,0.058116,0.042084,0.080160,0.058116,0.202405
4,Bromley,0.303435,0.028626,0.127863,0.085878,0.051527,0.070611,0.110687,0.055344,0.166031
5,Camden,0.452893,0.059504,0.125620,0.057851,0.082645,0.046281,0.066116,0.006612,0.102479
6,City of London,0.420635,0.063492,0.087302,0.063492,0.023810,0.119048,0.190476,0.000000,0.031746
7,Croydon,0.347826,0.014493,0.078675,0.072464,0.049689,0.053830,0.159420,0.053830,0.169772
8,Ealing,0.406137,0.032491,0.093863,0.063177,0.055957,0.061372,0.090253,0.054152,0.142599
9,Enfield,0.345114,0.024948,0.089397,0.076923,0.066528,0.064449,0.072765,0.054054,0.205821


In [5]:
grps = df_groups_norm.columns[1:].tolist()

In [9]:
{grp: 0 for grp in grps}

{'Eating out': 0,
 'Entertainment': 0,
 'Going out': 0,
 'Green spaces': 0,
 'Groceries': 0,
 'Health and Sports': 0,
 'Other': 0,
 'Public Transport': 0,
 'Shopping': 0}

In [18]:
ALL_VENUES = df_venues_raw.copy()
LONDON_GEOJSON = ldn_geojson
LONDON = [51.5074, -0.1278]

ALLOWED_ACM_TYPES = df_rent['Category'].unique().tolist()
ALLOWED_GROUPS = df_groups_norm.columns.tolist()[1:]

In [13]:
def filter_rent_data(df=df_rent, categories=None, rent_range=None):
    """
    Returns boroughs that satisfy the conditions for `categories` and `rent_range`
        
    Inputs:
        df - pandas DataFrame containing rent data -> !!! Assumes df_rent as default 
        categories - an iterable or a string specifying appropriate accommodation types
        rent_range - a list or a tuple with r_min and r_max rent ranges.
        
    Output:
        boroughs - a list of boroughs that match the condition
    
    """
    if isinstance(categories, str):
        cats = [categories]
    else:
        cats = categories
    
    cat_cond = df['Category'].isin(cats)
    rent_lower = rent_range[0]
    rent_higher = rent_range[1]
    
    # If invalid data provided
    if rent_lower > rent_higher:
        rent_higher = rent_lower
    
    rent_cond_1 = (df['Lower quartile'] <= rent_higher) & (df['Upper quartile'] >= rent_higher)
    rent_cond_2 = (df['Lower quartile'] <= rent_lower) & (df['Upper quartile'] >= rent_lower)
    rent_cond = rent_cond_1 | rent_cond_2
    rent_cond = (df['Lower quartile'] <= rent_higher) & (df['Upper quartile'] >= rent_lower)
    
    not_null = ~df['Median'].isnull()

    df_filtered = df.loc[(cat_cond & rent_cond & not_null)]
    
    boroughs = df_filtered['Borough'].unique().tolist()
   
    return boroughs

def create_preferences(ranking=None):
    "Converts `ranking` list to normalized pandas DataFrame"
    pref_dict = {
        'Eating out': 0,
        'Entertainment': 0,
        'Green spaces': 0,
        'Groceries': 0,
        'Health and Sports': 0,
        'Nightlife': 0,
        'Other': 0,
        'Public Transport': 0,
        'Shopping': 0,
    }
    
    N = len(ranking)
    
    for i, cat in enumerate(ranking): 
        pref_dict[cat] = N - i
    
    df = pd.DataFrame.from_dict(pref_dict, orient='index', columns=['Preference'])
    df['Preference'] = df['Preference'] / df['Preference'].sum()
    
    return df

def recommend_boroughs(W=df_groups_norm, p=None, n_brghs=5):
    """
    Calculates matrix product of DataFrames W and p
    
    Inputs:
        W - pandas DataFrame containing venue group density for each borough
        p - pandas DataFrame with user group preferences
        n_brghs - int, number of boroughs to recommend
    
    Returns:
        rec_boroughs - `n_brghs` number of boroughs by highest match value
        df_rec - resulting recommendation matrix as pandas DataFrame
    
    """
    W_ = W.copy()
    W_.set_index(['Borough'], inplace=True)
    df_rec = (p['Preference'] * W_).sum(axis=1).to_frame()
    df_rec.columns = ['Match']
    df_rec.sort_values(by='Match', ascending=False, inplace=True)
    rec_boroughs = df_rec.head(n_brghs).index.tolist()

    return rec_boroughs, df_rec


def recommend_and_plot(df_groups, df_rent, rent_range=None, acm_type=None, ranking=None, n=5):
    """
    Recommends top n venues and plots the results on the map
    
    """
    available_boroughs = filter_rent_data(df=df_rent, categories=acm_type, rent_range=rent_range)
    p = create_preferences(ranking=ranking)
    df_grp_filtered = df_groups.loc[df_groups['Borough'].isin(available_boroughs)]
    rec_boroughs, df_rec = recommend_boroughs(W=df_grp_filtered, p=p, n_brghs=n)
    
    
    df_rec.reset_index(inplace=True)
    
    df_all = ALL_VENUES.copy()
    df_matched = df_all.loc[(df_all['Borough'].isin(rec_boroughs)) & (df_all['Group'].isin(ranking))]
    
    print('='*80)
    print('= Recommended boroughs based on your preferences')
    print('='*80)
    
    for i, b in enumerate(rec_boroughs):
        print(f"#{i+1}: {b}")
    print('='*80)
    print('= Plotting venues...')
                                                                     
                             
    # Plotting
    map_rec = folium.Map(location=LONDON, tiles='cartodbpositron', zoom_start=10)

    # Chloropleth Map of boundaries, where shading is dependent on median rent for 'All categories'
    folium.Choropleth(
        geo_data=LONDON_GEOJSON,
        fill_color='BuPu',
        data=df_rec,
        columns=['Borough', 'Match'],
        key_on='feature.properties.name',
        weight=1,
        line_color='black',
        fill_opacity=0,
        line_opacity=0.5,
        highlight=True,
    ).add_to(map_rec)  

    df_boroughs = df_matched[['Borough', 'BoroughLat', 'BoroughLon']].drop_duplicates()
                             
    # Add borough markers to map
    for i, row in df_boroughs.iterrows():
        borough = row['Borough']
        coords = [row['BoroughLat'], row['BoroughLon']]
        rent_pref = f"{rent_range[0]} - {rent_range[1]}"
        lbl_str = f"{borough} ({rent_pref}\\xA3)" # create a label with borough name and rent range

        label = folium.Popup(lbl_str, parse_html=True) # so that the label is used as a Popup

        # Add the marker
        folium.CircleMarker(
            coords,
            radius=10,
            popup=label,
            color='black',
            weight=1,
            fill=True,
            fill_color='black',
            fill_opacity=0.75,
            parse_html=False,
        ).add_to(map_rec)  

    color_map = {
        'Eating out': '#e41a1c',
        'Entertainment': '#377eb8',
        'Going out': '#ffff33',
        'Green spaces': '#984ea3',
        'Groceries': '#ff7f00',
        'Health and Sports': '#4daf4a',
        'Other': '#999999',
        'Public Transport': '#a65628' ,
        'Shopping': '#f781bf',
    }

    for i, row in df_matched.iterrows():
#         # let's plot only every 5th point
#         if i % 10 == 0:
#             continue
        lat_i = row['Venue Latitude']
        lon_i = row['Venue Longitude']
        venue = row['Venue']
        group = row['Group']

        label = f"{venue} ({group})"

        coords = [lat_i, lon_i]
        folium.CircleMarker(
            coords,
            radius=4,
            popup=label,
            color='black',
            weight=1,
            fill=True,
            fill_color=color_map[group],
            fill_opacity=0.9,
            parse_html=False).add_to(map_rec)

    return map_rec

In [21]:
# Interactive stuff starts here!